# 导包

In [22]:
from collections import defaultdict
from tqdm import tqdm
import re
import json
from copy import deepcopy
import random
import torch
import sys,os
import pandas as pd
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample
from collections import Counter
from copy import deepcopy
from nlp_basictasks.modules.transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/')

2021-12-12 19:54:48 - INFO - from_pretrained - 125 : loading vocabulary file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/vocab.txt


# 加载数据

In [11]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/KgCLUE-main/qa_data/train.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/KgCLUE-main/qa_data/test_public.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/KgCLUE-main/qa_data/dev.json'))

原始数据有18000个样本
原始数据有2000个样本
原始数据有2000个样本


In [12]:
random.shuffle(data)
for i in range(10):
    print(data[i])

{'id': 7336, 'question': '胡秀英有什么成就吗？', 'answer': '胡秀英（重庆市万州区委党校经济学教研部主任） ||| 主要成就 ||| 2009年在重庆市委党校首次教学比赛中获二等奖'}
{'id': 26, 'question': '请问巴西海盆东西宽多少？', 'answer': '巴西海盆 ||| 东西宽 ||| 2000-2500公里'}
{'id': 17392, 'question': '潮汐海灵的魔法攻击是多少？', 'answer': '潮汐海灵（MOBA竞技网游《英雄联盟》中的英雄角色） ||| 魔法攻击 ||| 7'}
{'id': 5731, 'question': '我很好奇战地1943的总监是谁？', 'answer': '战地1943 ||| 总监 ||| GVD'}
{'id': 488, 'question': '请问山西华澳商贸职业学院的理事长是谁？', 'answer': '山西华澳商贸职业学院 ||| 理事长 ||| 刘耀国'}
{'id': 17464, 'question': '李波老家是哪的', 'answer': '李波（安徽省地震局党组成员、纪检组长） ||| 籍贯 ||| 山东省'}
{'id': 1360, 'question': '请问郎扎热甲节是哪年开始举行的？', 'answer': '郎扎热甲节 ||| 活动 ||| 熬上砖茶，做好酸奶'}
{'id': 1230, 'question': '2013第13届广州国际食品展暨广州进口食品展览会的展会名称是什么？', 'answer': '2013第13届广州国际食品展暨广州进口食品展览会 ||| 展会名称 ||| 2013第13届广州国际食品展暨广州进口食品展览会'}
{'id': 16831, 'question': '我想知道相对概念是属于什么系的？', 'answer': '相对概念 ||| 所属学科 ||| 逻辑学'}
{'id': 9753, 'question': '中国国际工程咨询有限公司的中文简称是什么啊？', 'answer': '中国国际工程咨询有限公司 ||| 中文简称 ||| 中国咨询、中咨公司'}


# 构造实体识别数据

In [13]:
labels=[]
sentences=[]
examples=[]
found_num=0
for example in data:
    question,answer=example['question'],example['answer']
    triplet=answer.strip().split('|||')
    assert len(triplet)==3
    entity=triplet[0]
    assert type(question)==str
    question=question.strip()
    found=False
    positions=[]
    entity=entity.strip()
    
    if '（' in entity and entity.endswith('）'):
        entity=entity.split('（')[0]
        
    if entity in question:
        start_id=question.find(entity)
        end_id=start_id+len(entity)
        positions.append((start_id,end_id))
        found=True
        
    if found:
        found_num+=1
        sentences.append(question)
        label=['O']*len(question)
        for each_span in positions:
            start_id,end_id=each_span
            for i in range(start_id,end_id):
                if i==start_id:
                    label[i]='B-ent'
                else:
                    label[i]='I-ent'
        labels.append(label)
        examples.append(example)

In [15]:
print(found_num,len(data))

21956 22000


In [16]:
print(len(sentences),len(labels))
for i in range(10):
    print(sentences[i],labels[i],examples[i])
    print('-'*100)

21956 21956
胡秀英有什么成就吗？ ['B-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 7336, 'question': '胡秀英有什么成就吗？', 'answer': '胡秀英（重庆市万州区委党校经济学教研部主任） ||| 主要成就 ||| 2009年在重庆市委党校首次教学比赛中获二等奖'}
----------------------------------------------------------------------------------------------------
请问巴西海盆东西宽多少？ ['O', 'O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 26, 'question': '请问巴西海盆东西宽多少？', 'answer': '巴西海盆 ||| 东西宽 ||| 2000-2500公里'}
----------------------------------------------------------------------------------------------------
潮汐海灵的魔法攻击是多少？ ['B-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 17392, 'question': '潮汐海灵的魔法攻击是多少？', 'answer': '潮汐海灵（MOBA竞技网游《英雄联盟》中的英雄角色） ||| 魔法攻击 ||| 7'}
----------------------------------------------------------------------------------------------------
我很好奇战地1943的总监是谁？ ['O', 'O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 5731,

# 构造训练集和测试集

In [21]:
index=list(range(len(sentences)))
random.shuffle(index)
test_nums=int(len(index)*0.1)
print(test_nums)
train_sentences=[]
train_labels=[]
train_raws=[]
for i in range(len(index)-test_nums):
    train_sentences.append(list(sentences[index[i]]))
    train_labels.append(labels[index[i]])
    train_raws.append(examples[index[i]])
    
k=10
print(train_sentences[k],train_labels[k],train_raws[k])

test_sentences=[]
test_labels=[]
test_raws=[]
for i in range(len(index)-test_nums,len(index)):
    test_sentences.append(list(sentences[index[i]]))
    test_labels.append(labels[index[i]])
    test_raws.append(examples[index[i]])
    
print(test_sentences[k],test_labels[k],test_raws[k])

2195
['你', '记', '得', '飓', '风', '什', '么', '时', '候', '上', '映', '的', '吗', '？'] ['O', 'O', 'O', 'B-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 10511, 'question': '你记得飓风什么时候上映的吗？', 'answer': '飓风（美国1999年诺曼·杰威森执导电影） ||| 上映时间 ||| 1999年12月29日'}
['D', 'u', 'l', 'l', 'a', 'h', 'a', 'n', '有', '什', '么', '弱', '点', '呀'] ['B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O'] {'id': 905, 'question': 'Dullahan有什么弱点呀', 'answer': 'Dullahan ||| 弱点 ||| 黄金'}


In [23]:
train_examples=[]
for seq_in,seq_out in zip(train_sentences,train_labels):
    train_examples.append(InputExample(seq_in=seq_in,seq_out=seq_out))

test_examples=[]
for seq_in,seq_out in zip(test_sentences,test_labels):
    test_examples.append(InputExample(seq_in=seq_in,seq_out=seq_out))

In [24]:
print(len(train_examples),len(test_examples))

19761 2195


In [25]:
label2id={'[PAD]':0,'O':1,'B-ent':2,'I-ent':3}

# 定义模型加载路径

In [26]:
model_path='/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/'
output_path='../NER'
ner_model=Ner(model_path,label2id=label2id,use_crf=True,use_bilstm=False,device='cuda',batch_first=True,
              tensorboard_logdir='../NER/log.txt')

2021-12-12 19:57:54 - INFO - __init__ - 53 : The label2id is
 {"[PAD]": 0, "O": 1, "B-ent": 2, "I-ent": 3}
2021-12-12 19:57:54 - INFO - __init__ - 270 : Loading model from /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/, which is from huggingface model
2021-12-12 19:57:54 - INFO - get_config_dict - 177 : loading configuration file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/config.json
2021-12-12 19:57:54 - INFO - from_pretrained - 404 : loading bert model file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/
2021-12-12 19:57:54 - INFO - from_pretrained - 423 : BertConfig has been loaded from /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/config.json
2021-12-12 19:57:55 - INFO - from_pretrained - 125 : loading vocabulary file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/vocab.txt
2021-12-12 19:57:55 - INFO - __init__ - 70 : Use pytorch device: cuda
2021-12-12 19:57:57 - INFO - __init__ - 72 : Using BiLSTM ? : False
2021-12-12 1

In [27]:
from torch.utils.data import DataLoader
batch_size=64
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

evaluator=nerEvaluator(label2id=label2id,seq_in=test_sentences,seq_out=test_labels,write_csv=True)

2021-12-12 19:58:04 - INFO - __init__ - 39 : Total evaluate nums : 2195
2021-12-12 19:58:04 - INFO - __init__ - 40 : input is string : False, input is list : True
2021-12-12 19:58:04 - INFO - __init__ - 41 : seq in and out like : 
['请', '问', '哪', '个', '公', '司', '举', '办', '了', '2', '0', '2', '0', '苹', '果', '秋', '季', '新', '品', '发', '布', '会', '？']	['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O']
2021-12-12 19:58:04 - INFO - __init__ - 42 : In this evaluator, slot contains ([PAD] O B-ent I-ent)


# 模型训练

In [28]:
ner_model.fit(train_dataloader=train_dataloader,evaluator=evaluator,epochs=5,output_path=output_path)

2021-12-12 19:58:07 - INFO - fit - 155 : label2id has been saved in ../NER/label2id.json
2021-12-12 19:58:07 - INFO - fit - 164 : 一个epoch 下，每隔61个step会输出一次loss，每隔154个step会评估一次模型


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-12-12 19:58:07 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:


  0%|          | 0/69 [00:00<?, ?it/s]

...

              precision    recall  f1-score   support

         ent     0.9877    0.9877    0.9877      2195

   micro avg     0.9877    0.9877    0.9877      2195
   macro avg     0.9877    0.9877    0.9877      2195
weighted avg     0.9877    0.9877    0.9877      2195



2021-12-12 20:00:50 - INFO - save_pretrained - 509 : Model weights saved in ../NER/BERT/pytorch_model.bin
2021-12-12 20:00:50 - INFO - save_pretrained - 150 : Configuration saved in ../NER/BERT/config.json
2021-12-12 20:00:50 - INFO - save_vocab - 51 : Vocab saved in ../NER/BERT/vocab.txt
2021-12-12 20:00:50 - INFO - fit - 239 : In epoch 4, training_step 770, the eval score is 0.9876993166287016, previous eval score is 0.9863387978142076, model has been saved in ../NER
2021-12-12 20:00:53 - INFO - fit - 217 : Epoch : 4, train_step : 915/1545, loss_value : 0.0008832592513320632 
2021-12-12 20:00:59 - INFO - fit - 217 : Epoch : 4, train_step : 1220/1545, loss_value : 0.0005409023140034195 
2021-12-12 20:01:05 - INFO - fit - 217 : Epoch : 4, train_step : 1525/1545, loss_value : 0.0013376651818436556 
2021-12-12 20:01:05 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 4 after 309 steps:


  0%|          | 0/69 [00:00<?, ?it/s]

2021-12-12 20:01:07 - INFO - fit - 243 : No improvement over previous best eval score (0.987474 vs 0.987699), patience = 4


              precision    recall  f1-score   support

         ent     0.9872    0.9877    0.9875      2195

   micro avg     0.9872    0.9877    0.9875      2195
   macro avg     0.9872    0.9877    0.9875      2195
weighted avg     0.9872    0.9877    0.9875      2195

